In [4]:
import warnings
import pandas as pd
import numpy as np
from scipy.linalg import inv, pinv, LinAlgError


# Framework

The system is considered in a steady-state


![static mfa](../img/static_mfa.jpg)  
Source: Lupton & Allwood (2018)

$\begin{cases}q_j + \sum\limits_{i=0}^nz_{ij} = \sum\limits_{i=0}^mz_{jk} \\ z_{jk} = \text{TC}_{jk} \begin{pmatrix}q_j + \sum\limits_{i=0}^nz_{ij}\end{pmatrix}\end{cases}$


# Utility functions

In [5]:
def get_data(filepath, xlsx_map):
    xlsx_file = pd.read_excel(filepath, sheet_name=None)
    TC = xlsx_map["transfer_coefficient"]
    FLOWS = xlsx_map["flows"]
    df_TC = xlsx_file[TC["sheet"]].rename(columns=xlsx_map["transfer_coefficient"])
    df_flows = xlsx_file[FLOWS["sheet"]].rename(columns=xlsx_map["flows"])
    return [df_TC, df_flows]


def solve(_in, _out, y, coeff, singular=False):
    """X = AX + y"""
    A = coeff[:, None] * (_in == _out[:, None])
    I = np.eye(len(A))
    L = pinv(I - A) if singular else inv(I - A)
    # VISUALIZE STEP
    print(f"b vector\n{y}\n\n")
    print(f"Technology matrix\n{I-A}\n\n")
    return L @ y


def solve_subsystem(df, *columns):
    _from, _to, _val, _pct = columns
    arr_from = df[_from].values
    arr_to = df[_to].values
    arr_pct = df[_pct].fillna(0).values
    arr_val = df[_val].fillna(0).values
    try:
        result = solve(_in=arr_to, _out=arr_from, y=arr_val, coeff=arr_pct)
    except LinAlgError as err:
        if "singular matrix" in str(err):
            idx = list(
                df.set_index(list(df.columns.difference(columns))).index.unique()
            )
            warnings.warn(
                f"singular matrix detected at {idx}. "
                "Defaulted on scipy.linalg.pinv for calculations"
            )
            result = solve(
                _in=arr_to, _out=arr_from, y=arr_val, coeff=arr_pct, singular=True
            )
        else:
            raise
    return list(zip(arr_from, arr_to, result))


def explode(df, *columns):
    _from, _to, _val, _pct = columns
    df = df.explode()
    return pd.DataFrame(
        data=df.values.tolist(),
        index=df.index,
        columns=[_from, _to, _val],
    )


def compute_system(filepath, xlsx_map, *columns):
    df_TC, df_flows = get_data(filepath, xlsx_map)
    other_columns = df_flows.columns.union(df_TC.columns).difference(columns)
    system_state = (
        pd.concat([df_flows, df_TC])
        .groupby(list(other_columns))
        .apply(solve_subsystem, *columns)
    )
    return explode(system_state, *columns)


# Application

In [6]:
FILEPATH = "../data/Test_sheet.xlsx"

FROM, TO, VAL, PCT = ("_from", "_to", "_val", "_pct")

XLSX_MAP = {
    # flows
    "flows": {
        "sheet": "input_flows",
        "Input_flow_name": FROM,
        "toProc": TO,
        "value": VAL,
    },
    # TCs
    "transfer_coefficient": {
        "sheet": "trans_coeff",
        "ProcessGen": FROM,
        "ProcessTo": TO,
        "Transfer_Coef": PCT,
    },
    # concentrations
    "concentration": {
        "sheet": "concentration",
        "Goods": FROM,
        "Substance": TO,
        "Concentration": PCT,
    },
}

compute_system(FILEPATH, XLSX_MAP, FROM, TO, VAL, PCT)


b vector
[0.01  0.5   0.1   0.04  0.1   0.003 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.   ]


Technology matrix
[[ 1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.  ]
 [ 0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.  ]
 [ 0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.  

_from    _to        _val
Country Material                          
EU      Cr        Flow1    BF1    0.010000
        Cr        Flow2    ENP    0.500000
        Cr        Flow3    ENP    0.100000
        Cr        Flow4    WIC    0.040000
        Cr        Flow5    WIC    0.100000
        Cr        Flow6    WIC    0.003000
        Cr          BF1   comm    0.006780
        Cr          BF1    RM1    0.474616
        Cr          BF1    RM2    0.196627
        Cr          SRM    BF1    0.668022
        Cr          ENP   comm    0.012000
        Cr          ENP  waste    0.108000
        Cr          ENP    RM3    0.480000
        Cr          WIC   comm    0.001430
        Cr          WIC    RM4    0.100100
        Cr          WIC  waste    0.041470
        Cr          RM1   comm    0.047462
        Cr          RM1  waste    0.189846
        Cr          RM1    SRM    0.237308
        Cr          RM2   comm    0.039325
        Cr          RM2  waste    0.078651
        Cr          RM2    SRM    0.078651
        Cr          RM3   comm    0.104008
        Cr          RM3  waste    0.104008
        Cr          RM3    SRM    0.312024
        Cr          RM4   comm    0.010010
        Cr          RM4  waste    0.010010
        Cr          RM4    RM3    0.040040
        Cr          RM4    SRM    0.040040
        Total     Flow1    BF1  130.200000
        Total     Flow2    ENP  223.500000
        Total     Flow3    ENP   22.100000
        Total     Flow4    WIC   44.500000
        Total     Flow5    WIC   10.500000
        Total     Flow6    WIC    3.200000
        Total       BF1   comm   14.692929
        Total       BF1    RM1  176.315143
        Total       BF1    RM2  102.850500
        Total       SRM    BF1  163.658571
        Total       ENP   comm   49.120000
        Total       ENP  waste  122.800000
        Total       ENP    RM3   73.680000
        Total       WIC   comm   40.740000
        Total       WIC    RM4   11.640000
        Total       WIC  waste    5.820000
        Total       RM1   comm   70.526057
        Total       RM1  waste   17.631514
        Total       RM1    SRM   88.157571
        Total       RM2   comm   20.570100
        Total       RM2  waste   41.140200
        Total       RM2    SRM   41.140200
        Total       RM3   comm   15.434400
        Total       RM3  waste   30.868800
        Total       RM3    SRM   30.868800
        Total       RM4   comm    2.328000
        Total       RM4  waste    2.328000
        Total       RM4    RM3    3.492000
        Total       RM4    SRM    3.492000

Current approach  
![current method](../img/current_method.png)

Original approach:  
![José method](../img/Jose_method.png)


In [ ]:
# FILEPATH = "../data/dummy_data.xlsx"

# FROM, TO, VAL, PCT = ("_from", "_to", "_val", "_pct")

# XLSX_MAP = {
#     # flows
#     "flows": {
#         "sheet": "input_flows",
#         "Input_flow_name": FROM,
#         "toProc": TO,
#         "value": VAL,
#     },
#     # TCs
#     "transfer_coefficient": {
#         "sheet": "trans_coeff",
#         "ProcessGen": FROM,
#         "ProcessTo": TO,
#         "Transfer_Coef": PCT,
#     },
#     # concentrations
#     "concentration": {
#         "sheet": "concentration",
#         "Goods": FROM,
#         "Substance": TO,
#         "Concentration": PCT,
#     },
# }

# compute_system(FILEPATH, XLSX_MAP, FROM, TO, VAL, PCT).tail(10)


In [ ]:
# FILEPATH = "../data/dummy_long.xlsx"

# FROM, TO, VAL, PCT = ("_from", "_to", "_val", "_pct")

# XLSX_MAP = {
#     # flows
#     "flows": {
#         "sheet": "inputs",
#         "processFrom": FROM,
#         "processTo": TO,
#         "value": VAL,
#     },
#     # TCs
#     "transfer_coefficient": {
#         "sheet": "trans_coeff",
#         "processFrom": FROM,
#         "processTo": TO,
#         "tc": PCT,
#     },
#     # concentrations
#     "concentration": {
#         "sheet": "concentration",
#         "Goods": FROM,
#         "Substance": TO,
#         "Concentration": PCT,
#     },
# }

# result = compute_system(FILEPATH, XLSX_MAP, FROM, TO, VAL, PCT)
# result


# References

Lupton, R. C., & Allwood, J. M. (2018). Incremental material flow analysis with Bayesian inference. Journal of Industrial Ecology, 22(6), 1352-1364.
